# Tensor与module的保存与加载

https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [2]:
import torch
from torch import nn
import os

# Tensor的保存与加载

In [3]:
# 序列化到文件
t = torch.randn(5, 5)
print(t)
torch.save(t, "/tmp/t.tensor")
t1 = torch.load("/tmp/t.tensor")
assert torch.sum(t - t1) < 1e-5
os.remove("/tmp/t.tensor")

tensor([[ 8.1307e-01,  7.7791e-02,  6.4913e-01,  5.2766e-01,  4.9626e-01],
        [-6.8846e-01,  3.0333e-01,  1.5317e-01,  9.0840e-01, -1.7860e+00],
        [-8.0498e-01, -5.7362e-01, -3.4265e-01, -1.3738e+00,  1.1405e+00],
        [-2.6447e-01,  9.4211e-01, -8.5745e-01, -1.9118e-01, -2.7557e-01],
        [-2.6007e-01,  1.9350e-03,  3.7661e-01,  1.2882e+00, -2.1934e+00]])


保存到内存中，以及从内存中加载

In [4]:
from io import BytesIO

t = torch.randn(5, 5)
buffer = BytesIO()
torch.save(t, buffer)
# 重置buffer的读写位置
buffer.seek(0)
t1 = torch.load(buffer)

# Module的保存与加载

可以单独保存模型的参数，也可以把整个模型保存起来

In [5]:
model = nn.Sequential(nn.Linear(25, 100), nn.ReLU(), nn.Linear(100, 10))
torch.save(model.state_dict(), "/tmp/mlp-params.pt")

In [6]:
params = torch.load("/tmp/mlp-params.pt")
# 现在的模型加载一份离线的参数
model.load_state_dict(params)

<All keys matched successfully>

直接保存整个 Moduel

In [8]:
torch.save(model, "/tmp/mlp-model.pt")

In [9]:
print(torch.load("/tmp/mlp-model.pt"))

Sequential(
  (0): Linear(in_features=25, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)


在 PyTorch 中，如果你直接保存整个模型模块（即调用 `torch.save(model, 'model.pth')`），它会尝试保存模型的所有内容，包括模型的结构、参数以及模型定义中用到的所有第三方库的引用。然而，这种方法并不保存第三方库的实际实现代码，只保存了对这些库的引用和调用。因此，当你在不同的环境中加载这个模型时，必须确保所有依赖的第三方库已经安装且版本兼容，否则可能会遇到问题。

仅保存 state_dict 可以确保模型在不同版本的 PyTorch 中更容易兼容。保存整个模型模块可能会导致在 PyTorch 更新版本后无法加载旧版本模型的问题，因为整个模型包含了版本相关的信息和代码。

# 使用GPU

In [10]:
gpu_model = model.to(device="cuda:0")

In [11]:
input = torch.randn(1, 25, device="cuda:0")
gpu_model(input)

tensor([[ 0.1331, -0.5196,  0.3243, -0.0070, -0.0491,  0.0294, -0.1772, -0.0146,
          0.2648, -0.3006]], device='cuda:0', grad_fn=<AddmmBackward0>)

GPU下保存的Tesnor或model，加载回来时，还是在对应的GPU上

In [12]:
t = torch.randn(3, 4, device="cuda")
torch.save(t, "/tmp/t-cuda.pt")
t = torch.load("/tmp/t-cuda.pt")
t.device

device(type='cuda', index=0)

# 优化器状态的保存与加载

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}

torch.save(checkpoint, "/tmp/tmp.ckpt")

In [15]:
checkpoint = torch.load("/tmp/tmp.ckpt")
print(checkpoint.keys())

dict_keys(['state_dict', 'optimizer'])


In [16]:
model.load_state_dict(checkpoint["state_dict"])
optimizer.load_state_dict(checkpoint["optimizer"])